Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [27]:
print('train_dataset', train_dataset.shape)
print('train_labels', train_labels.shape)
print('valid_dataset', valid_dataset.shape)
print('valid_labels', valid_labels.shape)
print('test_dataset', test_dataset.shape)
print('test_labels', test_labels.shape)
dims = train_dataset.shape[1]
hidden_layers = 128
num_classes = 10
print(dims)
batch_size = 128
reg = 0.005

def train_shit_mother_fucker(train_dataset, train_labels, valid_dataset, valid_labels, test_dataset, test_labels, train_dropput = 0.7):
    graph1 = tf.Graph()
    
    with graph1.as_default():
        X = tf.placeholder(tf.float32, [None, dims])
        y_labels = tf.placeholder(tf.float32, [None, num_classes])
        keep_prob_tensor = tf.placeholder(tf.float32)

        W1 = tf.Variable(tf.random_uniform([784, hidden_layers], -0.01, 0.01), dtype=tf.float32)
        b1 = tf.Variable(tf.zeros([hidden_layers]), dtype=tf.float32)

        z1 = tf.matmul(X, W1) + b1
        a1 = tf.nn.relu(z1)
        
        drop_out_layer1 = tf.nn.dropout(a1, keep_prob_tensor)

        W2 = tf.Variable(tf.random_uniform([hidden_layers, num_classes], -0.01, 0.01), dtype=tf.float32)
        b2 = tf.Variable(tf.zeros([num_classes]), dtype=tf.float32)

        z2 = tf.matmul(drop_out_layer1, W2) + b2
        a2 = tf.nn.softmax(z2)

        regularizer = reg * tf.nn.l2_loss(W2) + reg * tf.nn.l2_loss(W1)
        loss1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_labels, logits=a2))
        loss1 = tf.reduce_mean(loss1 + regularizer)
        optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss1)

        with tf.Session(graph=graph1) as sess:
            l_value = None

            tf.global_variables_initializer().run()
            for i in range(20000):
                offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = train_dataset[offset:(offset + batch_size), :]
                #print('train_labels', train_labels.shape)
                batch_labels = train_labels[offset:(offset + batch_size), :]
                feed_dict={
                    X:batch_data, 
                    y_labels: batch_labels,
                    keep_prob_tensor: train_dropput
                }
                _, l_value = sess.run([optimizer, loss1], feed_dict=feed_dict)
                if i % 1000 == 0:
                    print(i, ' : ', l_value)
            print("Final Loss", l_value)

            correct_prediction = tf.equal(tf.argmax(y_labels,1), tf.argmax(a2,1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            print('val acc: ', sess.run(accuracy, feed_dict={X: valid_dataset, y_labels: valid_labels,
                    keep_prob_tensor: 1}))
            print('test acc: ', sess.run(accuracy, feed_dict={X: test_dataset, y_labels: test_labels,
                    keep_prob_tensor: 1}))



train_dataset (200000, 784)
train_labels (200000, 10)
valid_dataset (10000, 784)
valid_labels (10000, 10)
test_dataset (10000, 784)
test_labels (10000, 10)
784


In [28]:
train_shit_mother_fucker(train_dataset, train_labels, valid_dataset, valid_labels, test_dataset, test_labels)

0  :  2.311045
1000  :  1.7945662
2000  :  1.7664502
3000  :  1.7479265
4000  :  1.8122343
5000  :  1.7545754
6000  :  1.7294809
7000  :  1.7511207
8000  :  1.7768226
9000  :  1.7315966
10000  :  1.735817
11000  :  1.7778652
12000  :  1.7473674
13000  :  1.7281364
14000  :  1.7516723
15000  :  1.721434
16000  :  1.7041763
17000  :  1.734987
18000  :  1.7418755
19000  :  1.7202544
Final Loss 1.7296227
val acc:  0.8384
test acc:  0.8951


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [32]:
train_samples_count = batch_size * 2
train_shit_mother_fucker(train_dataset[:train_samples_count, :], train_labels[:train_samples_count, :], valid_dataset, valid_labels, test_dataset, test_labels, 1)

0  :  2.3110566
1000  :  1.5738357
2000  :  1.5638618
3000  :  1.562144
4000  :  1.5612178
5000  :  1.5605485
6000  :  1.5600239
7000  :  1.5595995
8000  :  1.5592484
9000  :  1.5589606
10000  :  1.5587075
11000  :  1.5584935
12000  :  1.5583131
13000  :  1.5581363
14000  :  1.5579823
15000  :  1.5578573
16000  :  1.5577476
17000  :  1.5576476
18000  :  1.557559
19000  :  1.557477
Final Loss 1.5574032
val acc:  0.719
test acc:  0.7807


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
